### 1) Setup (paths + imports)

In [9]:
import numpy as np
import matplotlib.pyplot as plt
import joblib
from pathlib import Path
import sys, os
import joblib


In [10]:
REPO_ROOT = os.path.abspath(os.path.join(os.getcwd(), ".."))
if REPO_ROOT not in sys.path:
    sys.path.insert(0, REPO_ROOT)
MODELS_DIR = Path(REPO_ROOT) / "models"
MODEL_PATH = MODELS_DIR / "automl_best_one.pkl"
LE_PATH    = MODELS_DIR / "label_encoder.pkl"
IMAGES_DIR = Path(REPO_ROOT) / "sample_images"
SCALER_PATH = MODELS_DIR / "scaler.pkl"  
from src.extract_features import extract_enhanced_features


### 2) Load model + label encoder

In [11]:
model = joblib.load(MODEL_PATH)
scaler = joblib.load(SCALER_PATH)
le    = joblib.load(LE_PATH)




### 3) Collect images to test


In [12]:
# Recursively collect image paths (jpg/png/tif…)
exts = {".jpg",".jpeg",".png",".bmp",".tif",".tiff"}
all_imgs = [p for p in IMAGES_DIR.rglob("*") if p.suffix.lower() in exts]



In [15]:
all_imgs = [p for p in IMAGES_DIR.rglob("*") if p.suffix.lower() in exts]

if len(all_imgs) == 0:
    raise FileNotFoundError(
        f"No images found in {IMAGES_DIR}. "
        "Add a few test images under sample_images/<class_name>/image.jpg"
    )
test_paths = all_imgs

In [16]:
# Just extract features - no manual scaling!
def extract_features(image_paths):
    X = [extract_enhanced_features(p) for p in image_paths]
    return np.array(X, dtype=np.float32)

# Extract features
X_test = extract_features(test_paths)

# Pipeline handles scaling + SMOTE + prediction internally
y_pred = model.predict(X_test)
y_pred_proba = model.predict_proba(X_test)
pred_labels = le.inverse_transform(y_pred)
confidences = np.max(y_pred_proba, axis=1)

# Display results
for i, (img_path, pred_label, conf) in enumerate(zip(test_paths, pred_labels, confidences), 1):
    print(f"[{i}] {img_path.name}: {pred_label} ({conf:.1%})")

[1] basophil.jpg: Basophil (100.0%)
[2] esinophil.jpg: Lymphocyte (42.8%)
[3] lymphocyte.jpg: Monocyte (38.6%)
[4] monocyte.jpg: Monocyte (88.2%)
[5] neutrophil.jpg: Neutrophil (59.8%)
